In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import plot
import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz
scales = np.array([1000, 2000, 4000, 8000, 16000, 25000])

# r_kpc = 1
# heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
# # scales = heights_and_scales['scales'] # AU
# heights = heights_and_scales['heights'] # AU  

# colors = plot.COLORS

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array([4000, 9000, 21000, 39000, 54000, 68000, 94000])
# years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
# masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
#                    r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
#                    r'24 M$_\odot$'])
masss = np.array([1.4, 2, 4, 8, 12, 16, 24])



file = np.load(VICO_loc+'/AngladaRosero2019/AngladaData/shock_flux_arrays.npz')
lums = file['luminosities']
file.close()

In [ ]:
print(lums)

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = basic_snapshot(snap=num, name = ('Snap%03d_n' % num))
    snaps[ii].load_variabilities()

In [ ]:
def plot_flux_vs_mass(aa, log_flag=True):
    ylabel=r'Flux, $S_\nu$ (mJy)'
    labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                    '16,000 au', '25,000 au'])
    dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
    colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
    # ylim0 = (10**-1.5, 10**3.5)

    # ff_arr = [5, 9]
    fig, axs = plot.figax_double(ncols=2, nrows=2, height = 7, xscale='linear', sharey=True,)

    avg_str = np.array(['averages', 'averages_const'])
    std_str = np.array(['stdevs', 'stdevs_ratio'])

    y1 = np.zeros((len(scales), len(snaps)))
    err1 = np.zeros_like(y1)
    y2 = np.zeros((len(scales), len(snaps)))
    err2 = np.zeros_like(y2)
    for ss, snap in enumerate(snaps):
        # print(ss)
        y1[:,ss]   = 10**snap.Variabilities[avg_str[aa]][5,1:]
        err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
        y2[:,ss]   = 10**snap.Variabilities[avg_str[aa]][9,1:]
        err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

    # set errorbars
    if log_flag:
        ymin1 = 10**(np.log10(y1) - np.log10(err1))
        ymax1 = 10**(np.log10(y1) + np.log10(err1))
        ymin2 = 10**(np.log10(y2) - np.log10(err2))
        ymax2 = 10**(np.log10(y2) + np.log10(err2))
    else:
        ymin1 = y1 - err1
        ymax1 = y1 + err1
        ymin2 = y2 - err2
        ymax2 = y2 + err2


    fflabels = ['5.3 GHz', '230 GHz']

    ##################################
    ####### first column, vs Mass ######
    ##################################
    xlabel='$m_*\ (\mathrm{M}_\odot)$'
    ylabels = [r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[5]/10**9),
                r'$S_\nu$ (%s GHz) [mJy]' % str(freqs[9]/10**9)]
    axs[1,0].sharex(axs[0,0])

    xx = masss
    for ii,ax in enumerate(axs[:,0]): ax.set_ylabel(ylabels[ii])
    axs[1,0].set_xlabel(xlabel)

    for ss, scale in enumerate(scales):
        label = labels[ss] #if aa==1 else None
        axs[0,0].errorbar(xx, y1[ss], yerr=(y1[ss]-ymin1[ss], ymax1[ss]-y1[ss]), color=colors[ss],
                            capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
        axs[1,0].errorbar(xx, y2[ss], yerr=(y2[ss]-ymin2[ss], ymax2[ss]-y2[ss]), color=colors[ss],
                            capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='-')


    # leg1 = ax.legend(loc='lower right', ncol=2)
    # legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
    #                    Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
    # leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 0), loc='lower right')
    # ax.add_artist(leg1)



    ########################################
    ####### left panel, vs Luminosity ######
    ########################################
    ax = axs[1]
    xlabel='$L_\mathrm{bol} (\mathrm{L}_\odot)$'
    xx = lums

    axs[1,1].set_xlabel(xlabel)
    axs[1,1].sharex(axs[0,1])
    axs[0,1].set_xscale('log')

    for ii, scale in enumerate(scales):
        label = labels[ii] #if aa==1 else None
        axs[0,1].errorbar(xx, y1[ii], yerr=(y1[ii]-ymin1[ii], ymax1[ii]-y1[ii]), color=colors[ii],
                            capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
        axs[1,1].errorbar(xx, y2[ii], yerr=(y2[ii]-ymin2[ii], ymax2[ii]-y2[ii]), color=colors[ii],
                            capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')



    leg1 = axs[0,0].legend(loc='upper right', ncol=2)
    fig.tight_layout()
    return fig

fig1 = plot_flux_vs_mass(aa=0, log_flag=True)
fig1.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseA_logerr.png', dpi=300)

fig2 = plot_flux_vs_mass(aa=1, log_flag=True)
fig2.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseB_logerr.png', dpi=300)

fig1 = plot_flux_vs_mass(aa=0, log_flag=False)
fig1.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseA_linerr.png', dpi=300)

fig2 = plot_flux_vs_mass(aa=1, log_flag=False)
fig2.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseB_linerr.png', dpi=300)

In [ ]:
ylabel=r'Flux, $S_\nu$ (mJy)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
# ylim0 = (10**-1.5, 10**3.5)

# ff_arr = [5, 9]
fig, axs = plot.figax_double(ncols=2, height = 4, xscale='linear', sharey=True,)

avg_str = np.array(['averages', 'averages_const'])
std_str = np.array(['stdevs', 'stdevs_ratio'])
aa=0

y1 = np.zeros((len(scales), len(snaps)))
err1 = np.zeros_like(y1)
y2 = np.zeros((len(scales), len(snaps)))
err2 = np.zeros_like(y2)
for ss, snap in enumerate(snaps):
    # print(ss)
    y1[:,ss]   = 10**snap.Variabilities[avg_str[aa]][5,1:]
    err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
    y2[:,ss]   = 10**snap.Variabilities[avg_str[aa]][9,1:]
    err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

# set errorbars
ymin1 = 10**(np.log10(y1) - np.log10(err1))
ymax1 = 10**(np.log10(y1) + np.log10(err1))
ymin2 = 10**(np.log10(y2) - np.log10(err2))
ymax2 = 10**(np.log10(y2) + np.log10(err2))

fflabels = ['5.3 GHz', '230 GHz']

##################################
####### left panel, vs Mass ######
##################################
ax = axs[0]
xlabel='Mass ($\mathrm{M}_\odot$)'
xx = masss

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

for ii, scale in enumerate(scales):
    label = labels[ii] #if aa==1 else None
    ax.errorbar(xx, y1[ii], yerr=err1[ii], color=colors[ii],
                        capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
    ax.errorbar(xx, y2[ii], yerr=err2[ii], color=colors[ii],
                        capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
# axs[0,0].set_ylim(ylim0)


# leg1 = ax.legend(loc='lower right', ncol=2)
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 0), loc='lower right')
# ax.add_artist(leg1)



########################################
####### left panel, vs Luminosity ######
########################################
ax = axs[1]
xlabel='Bolometric Luminosity $ (\mathrm{L}_\odot$)'
xx = lums

ax.set_xlabel(xlabel)
ax.set_xscale('log')

for ii, scale in enumerate(scales):
    label = labels[ii] #if aa==1 else None
    ax.errorbar(xx, y1[ii], yerr=err1[ii], color=colors[ii],
                        capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
    ax.errorbar(xx, y2[ii], yerr=err2[ii], color=colors[ii],
                        capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
# axs[0,0].set_ylim(ylim0)
# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])
leg1 = ax.legend(loc='lower right', ncol=2)
# legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
#                    Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
# leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 1), loc='upper right')
# ax.add_artist(leg1)


fig.tight_layout()
# fig.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseA_linerr.png', dpi=300)


In [ ]:
print(err1)

In [ ]:
ylabel=r'Flux, $S_\nu$ (mJy)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
# ylim0 = (10**-1.5, 10**3.5)

# ff_arr = [5, 9]
fig, axs = plot.figax_double(ncols=2, height = 4, xscale='linear', sharey=True,)

avg_str = np.array(['averages', 'averages_const'])
std_str = np.array(['stdevs', 'stdevs_ratio'])
aa=1

y1 = np.zeros((len(scales), len(snaps)))
err1 = np.zeros_like(y1)
y2 = np.zeros((len(scales), len(snaps)))
err2 = np.zeros_like(y2)
for ss, snap in enumerate(snaps):
    # print(ss)
    y1[:,ss]   = 10**snap.Variabilities[avg_str[aa]][5,1:]
    err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
    y2[:,ss]   = 10**snap.Variabilities[avg_str[aa]][9,1:]
    err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

# set errorbars
ymin1 = 10**(np.log10(y1) - np.log10(err1))
ymax1 = 10**(np.log10(y1) + np.log10(err1))
ymin2 = 10**(np.log10(y2) - np.log10(err2))
ymax2 = 10**(np.log10(y2) + np.log10(err2))


fflabels = ['5.3 GHz', '230 GHz']

##################################
####### left panel, vs Mass ######
##################################
ax = axs[0]
xlabel='Mass ($\mathrm{M}_\odot$)'
xx = masss

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

for ii, scale in enumerate(scales):
    label = labels[ii] #if aa==1 else None
    ax.errorbar(xx, y1[ii], yerr=(y1[ii]-ymin1[ii], ymax1[ii]-y1[ii]), color=colors[ii],
                        capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
    ax.errorbar(xx, y2[ii], yerr=(y2[ii]-ymin2[ii], ymax2[ii]-y2[ii]), color=colors[ii],
                        capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')

leg1 = ax.legend(loc='upper right', ncol=2, columnspacing=1.0 )
# legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
#                    Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
# leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 1), loc='upper right')
# ax.add_artist(leg1)


########################################
####### left panel, vs Luminosity ######
########################################
ax = axs[1]
xlabel='Bolometric Luminosity $ (\mathrm{L}_\odot$)'
xx = lums

ax.set_xlabel(xlabel)
ax.set_xscale('log')

for ii, scale in enumerate(scales):
    label = labels[ii] #if aa==1 else None
    ax.errorbar(xx, y1[ii], yerr=(y1[ii]-ymin1[ii], ymax1[ii]-y1[ii]), color=colors[ii],
                        capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
    ax.errorbar(xx, y2[ii], yerr=(y2[ii]-ymin2[ii], ymax2[ii]-y2[ii]), color=colors[ii],
                        capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])

# leg1 = ax.legend(loc='lower right', ncol=2)
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 1), loc='upper right')
# ax.add_artist(leg1)


fig.tight_layout()
# fig.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseB_logerr.png', dpi=300)


In [ ]:
ylabel=r'Flux, $S_\nu$ (mJy)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
# ylim0 = (10**-1.5, 10**3.5)

# ff_arr = [5, 9]
fig, axs = plot.figax_double(ncols=2, height = 4, xscale='linear', sharey=True,)

avg_str = np.array(['averages', 'averages_const'])
std_str = np.array(['stdevs', 'stdevs_ratio'])
aa=1

y1 = np.zeros((len(scales), len(snaps)))
err1 = np.zeros_like(y1)
y2 = np.zeros((len(scales), len(snaps)))
err2 = np.zeros_like(y2)
for ss, snap in enumerate(snaps):
    # print(ss)
    y1[:,ss]   = 10**snap.Variabilities[avg_str[aa]][5,1:]
    err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
    y2[:,ss]   = 10**snap.Variabilities[avg_str[aa]][9,1:]
    err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

# set errorbars
ymin1 = 10**(np.log10(y1) - np.log10(err1))
ymax1 = 10**(np.log10(y1) + np.log10(err1))
ymin2 = 10**(np.log10(y2) - np.log10(err2))
ymax2 = 10**(np.log10(y2) + np.log10(err2))


fflabels = ['5.3 GHz', '230 GHz']

##################################
####### left panel, vs Mass ######
##################################
ax = axs[0]
xlabel='Mass ($\mathrm{M}_\odot$)'
xx = masss

ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)

for ii, scale in enumerate(scales):
    label = labels[ii] #if aa==1 else None
    ax.errorbar(xx, y1[ii], yerr=err1[ii], color=colors[ii],
                        capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
    ax.errorbar(xx, y2[ii], yerr=err2[ii], color=colors[ii],
                        capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
# axs[0,0].set_ylim(ylim0)

leg1 = ax.legend(loc='upper right', ncol=2, columnspacing=1.0 )
# legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
#                    Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
# leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 1), loc='upper right')
# ax.add_artist(leg1)


########################################
####### left panel, vs Luminosity ######
########################################
ax = axs[1]
xlabel='Bolometric Luminosity $ (\mathrm{L}_\odot$)'
xx = lums

ax.set_xlabel(xlabel)
ax.set_xscale('log')

for ii, scale in enumerate(scales):
    label = labels[ii] #if aa==1 else None
    ax.errorbar(xx, y1[ii], yerr=err1[ii], color=colors[ii],
                        capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
    ax.errorbar(xx, y2[ii], yerr=err2[ii], color=colors[ii],
                        capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])

# leg1 = ax.legend(loc='lower right', ncol=2)
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
leg2 = ax.legend(handles=legend_elements, bbox_to_anchor=(1, 1), loc='upper right')
# ax.add_artist(leg1)


fig.tight_layout()
# fig.savefig(VICO_loc+'/figures/flux_vs_mass/flux_vs_mass_caseB_linerr.png', dpi=300)


In [ ]:
ylabel=r'Flux, $S_\nu$ (mJy)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
# ylim0 = (10**-1.5, 10**3.5)

# ff_arr = [5, 9]
fig, axs = plot.figax(figsize = (11,4), ncols=2,
    xlabel='Years', sharex=True, xscale='linear',)
# for ax in axs[:,0]:
axs[0].set_ylabel(ylabel)

avg_str = np.array(['averages', 'averages_const'])
std_str = np.array(['stdevs', 'stdevs_ratio'])

for aa, ax in enumerate(axs):
    xx = years
    y1 = np.zeros((len(scales), len(snaps)))
    err1 = np.zeros_like(y1)
    y2 = np.zeros((len(scales), len(snaps)))
    err2 = np.zeros_like(y2)
    for ss, snap in enumerate(snaps):
        # print(ss)
        y1[:,ss]   = 10**snap.Variabilities[avg_str[aa]][5,1:]
        err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
        y2[:,ss]   = 10**snap.Variabilities[avg_str[aa]][9,1:]
        err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

    fflabels = ['5.3 GHz', '230 GHz']

    for ii, scale in enumerate(scales):
        label = labels[ii] #if aa==1 else None
        ax.errorbar(xx, y1[ii], yerr=err1[ii], color=colors[ii],
                         capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
        ax.errorbar(xx, y2[ii], yerr=err2[ii], color=colors[ii],
                         capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
    axtext = ['no cooling', 'with cooling']
    for ii, ax in enumerate(axs):
        ax.text(0.99,0.95, axtext[ii], transform=ax.transAxes, weight='bold', horizontalalignment='right')
# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])
axs[0].legend(loc='lower right')
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
axs[1].legend(handles=legend_elements, bbox_to_anchor=(1, 0.95), loc='upper right')


fig.tight_layout()
fig.savefig(VICO_loc+'/figures/flux_vs_time.png', dpi=300)


# Variability vs Time

In [ ]:
ylabel=r'Flux Variability, $\langle \Delta S_\nu / S_\nu \rangle$'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
# dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
# colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
# ylim0 = (10**-1.5, 10**3.5)

# ff_arr = [5, 9]
fig, axs = plot.figax(figsize = (11,4), ncols=2,
    xlabel='Years', sharex=True, xscale='linear',)
# for ax in axs[:,0]:
axs[0].set_ylabel(ylabel)

dif_str = np.array(['difsavg', 'difsavg_ratio'])
# std_str = np.array(['stdevs', 'stdevs_ratio'])

for aa, ax in enumerate(axs):
    xx = years
    y1 = np.zeros((len(scales), len(snaps)))
    err1 = np.zeros_like(y1)
    y2 = np.zeros((len(scales), len(snaps)))
    err2 = np.zeros_like(y2)
    for ss, snap in enumerate(snaps):
        # print(ss)
        y1[:,ss]   = snap.Variabilities[dif_str[aa]][5,1:]
        # err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
        y2[:,ss]   = snap.Variabilities[dif_str[aa]][9,1:]
        # err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

    fflabels = ['5.3 GHz', '230 GHz']

    for ii, scale in enumerate(scales):
        label = labels[ii] #if aa==1 else None
        ax.plot(xx, y1[ii], color=colors[ii],
                         label=label, alpha=0.75, linewidth=2, linestyle='-', marker='o',)
        ax.plot(xx, y2[ii], color=colors[ii],
                         label=None, alpha=0.75, linewidth=2, linestyle='--', marker='+')
    axtext = ['no cooling', 'with cooling']
    for ii, ax in enumerate(axs):
        ax.text(0.99,0.95, axtext[ii], transform=ax.transAxes, weight='bold', horizontalalignment='right')
# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])
axs[0].legend(loc='upper left')
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
axs[1].legend(handles=legend_elements, bbox_to_anchor=(1, 0.95), loc='upper right')

fig.tight_layout()
# fig.savefig(VICO_loc+'/figures/flux_vs_time.png', dpi=300)


In [ ]:
print(y1 - y2)

Add de Pree lines

In [ ]:
# # De Pree et al
# depree_lines = np.array([6.862745098, 5.04587156, 21.05263158,
#                             2.53164557, 5.303030303, 38.53211009,
#                             2.564102564, 8.045977011, 4.87804878])
# depree_sources = np.array(['B', 'C', 'C1', 
#                             'D', 'F', 'G2', 
#                             'G3', 'G4', 'G5'])
# depree_xtext = np.array([20000,20000,15000,
#                         15000,15000,15000,
#                         15000,15000,25000])

dplines = np.array([2.53164557, 2.564102564, 4.87804878,
                    5.04587156, 5.303030303, 6.862745098, 
                    8.045977011, 21.05263158, 38.53211009,])
dpnames = np.array(['D', 'G3', 'G5',
                    'C', 'F', 'B',
                     'G4', 'C1', 'G2' ])
dp_xpos = np.array([26000, 20000, 27000,
                    23500, 20000, 17000, 
                    11000, 10000, 9000])


In [ ]:
def plot_variability(case='A'):

    ylabel=r'Flux Variability, $\langle \Delta S_\nu / S_\nu \rangle$ (%)'
    labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                    '16,000 au', '25,000 au'])

    # ff_arr = [5, 9]
    fig, axs = plot.figax_single(nrows=2, height=7,
        xscale='linear', sharex=True)

    ylabels = ['5.3 GHz Flux Variability (%)', '230 GHz Flux Variability (%)']
    for ii, ax in enumerate(axs):
        ax.set_ylabel(ylabels[ii])
    xlabel='Years'
    axs[1].set_xlabel(xlabel)

    dif_str = np.array(['difsavg', 'difsavg_ratio'])

    xx = years
    y1 = np.zeros((len(scales), len(snaps)))
    err1 = np.zeros_like(y1)
    y2 = np.zeros((len(scales), len(snaps)))
    err2 = np.zeros_like(y2)

    if case=='A':
        aa = 0
    elif case=='B':
        aa = 1
    else:
        err = f"{case=} must be 'A' or 'B'"
        raise ValueError(err)
    
    # Make y arrays
    for ss, snap in enumerate(snaps):
        # print(ss)
        y1[:,ss]   = snap.Variabilities[dif_str[aa]][5,1:]*100
        # err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
        y2[:,ss]   = snap.Variabilities[dif_str[aa]][9,1:]*100
        # err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

    # Plot
    for ii, scale in enumerate(scales):
        label = labels[ii] #if aa==1 else None
        axs[0].plot(xx, y1[ii], color=colors[ii],
                        label=label, alpha=0.9, linewidth=1.5, linestyle='-',)
        axs[1].plot(xx, y2[ii], color=colors[ii],
                        label=label, alpha=0.9, linewidth=2, linestyle='-',)
        

    # Add de Pree lines   
    for ll, line in enumerate(dplines):
        axs[0].axhline(y=line, color='k', linestyle=':', linewidth=1.2, alpha=0.5)
        axs[0].text(dp_xpos[ll], dplines[ll]*0.92, dpnames[ll], fontsize=12)



    # axs[0,0].set_ylim(ylim0)
    # axs[0,1].sharey(axs[0,0])
    legend_elements = [Line2D([0], [0], color='k', lw=1.2, linestyle=':', label='W49A (8.3 GHz)')]
    loc = 'lower right' if case=='A' else 'lower left'
    axs[0].legend(handles=legend_elements, loc=loc)
    axs[1].legend(loc=loc, ncols=2)


    fig.tight_layout()
    return fig


fig1 = plot_variability(case='A')
fig1.savefig(VICO_loc+'/figures/flux_variability_caseA.png', dpi=300, bbox_inches='tight')

fig2 = plot_variability(case='B')
fig2.savefig(VICO_loc+'/figures/flux_variability_caseB.png', dpi=300)

In [ ]:
ylabel=r'Flux Variability, $\langle \Delta S_\nu / S_\nu \rangle$ (%)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])

# percen = 100 # if percent

# ff_arr = [5, 9]
fig, axs = plot.figax(figsize = (11,4), ncols=2,
    xlabel='Years', sharex=True, xscale='linear', sharey=True)
# for ax in axs[:,0]:
axs[0].set_ylabel(ylabel)

dif_str = np.array(['difsavg', 'difsavg_ratio'])
# std_str = np.array(['stdevs', 'stdevs_ratio'])

for aa, ax in enumerate(axs):
    xx = years
    y1 = np.zeros((len(scales), len(snaps)))
    err1 = np.zeros_like(y1)
    y2 = np.zeros((len(scales), len(snaps)))
    err2 = np.zeros_like(y2)
    for ss, snap in enumerate(snaps):
        # print(ss)
        y1[:,ss]   = snap.Variabilities[dif_str[aa]][5,1:]*100
        # err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
        y2[:,ss]   = snap.Variabilities[dif_str[aa]][9,1:]*100
        # err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

    fflabels = ['5.3 GHz', '230 GHz']

    for ii, scale in enumerate(scales):
        label = labels[ii] #if aa==1 else None
        ax.plot(xx, y1[ii], color=colors[ii],
                         label=label, alpha=0.75, linewidth=1.5, linestyle='-', marker='o',)
        ax.plot(xx, y2[ii], color=colors[ii],
                         label=None, alpha=0.75, linewidth=2, linestyle='--', marker='+')
    axtext = ['no cooling', 'with cooling']
    for ii, ax in enumerate(axs):
        ax.text(0.99,0.95, axtext[ii], transform=ax.transAxes, weight='bold', horizontalalignment='right')
for ax in axs:
    for ll, line in enumerate(dplines):
        ax.axhline(y=line, color='k', linestyle=':', linewidth=1.2, alpha=0.5)
        ax.text(dp_xpos[ll], dplines[ll]*0.92, dpnames[ll], fontsize=12)



# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])
axs[1].legend(loc='upper right', bbox_to_anchor=(1, 0.95), )
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz'),
                   Line2D([0], [0], color='k', lw=1.2, linestyle=':', label='W49A (8.3 GHz)')]
axs[0].legend(handles=legend_elements, loc='lower right')


fig.tight_layout()
fig.savefig(VICO_loc+'/figures/flux_variability.png', dpi=300)
